In [ ]:
# recommond to reload this file on colab
!git clone -l -s https://github.com/asteroid-team/asteroid.git asteroid

Cloning into 'asteroid'...
remote: Enumerating objects: 7684, done.
remote: Counting objects: 100% (307/307), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 7684 (delta 143), reused 248 (delta 111), pack-reused 7377
Receiving objects: 100% (7684/7684), 5.98 MiB | 15.63 MiB/s, done.
Resolving deltas: 100% (4836/4836), done.


In [ ]:
%cd /content/asteroid

/content/asteroid


In [ ]:
! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 12.5 MB/s 
     |████████████████████████████████| 798 kB 58.2 MB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 59 kB 8.1 MB/s 
     |████████████████████████████████| 125 kB 77.5 MB/s 
     |████████████████████████████████| 512 kB 79.2 MB/s 
     |████████████████████████████████| 87 kB 8.1 MB/s 
     |████████████████████████████████| 41 kB 565 kB/s 
     |████████████████████████████████| 90 kB 12.2 MB/s 
  Created wheel for torch-stoi: filename=torch_stoi-0.1.2-py3-none-any.whl size=6200 sha256=ef8bedc1495e8eba0cd587272ca4ae732937ab39c216dbf913f08fbf4d7ade12
  Stored in directory: /root/.cache/pip/wheels/55/96/76/4e46c2df4cfd5c6411d5d18bb46dd52552bdce0df460f94dc0
  Created wheel for julius: filename=julius-0.2.7-py3-none-any.whl size=21895 sha256=969bb0f0ff59e6c32c5d9611c52699154e08594

## Reload own trained model

In [ ]:
# Asteroid is based on PyTorch and PyTorch-Lightning.
from torch import optim
from pytorch_lightning import Trainer

# We train the same model architecture that we used for inference above.
from asteroid.models import DPRNNTasNet
#from asteroid.models import DPTNet

# In this example we use Permutation Invariant Training (PIT) and the SI-SDR loss.
from asteroid.losses import pairwise_neg_sisdr, PITLossWrapper

# MiniLibriMix is a tiny version of LibriMix (https://github.com/JorisCos/LibriMix),
# which is a free speech separation dataset.
from asteroid.data import LibriMix

# Asteroid's System is a convenience wrapper for PyTorch-Lightning.
from asteroid.engine import System

# This will automatically download MiniLibriMix from Zenodo on the first run.
train_loader, val_loader = LibriMix.loaders_from_mini(task="sep_noisy", batch_size=16, n_src=2)

# Tell DPRNN that we want to separate to 2 sources.
model = DPRNNTasNet(n_src=2)
#newmodel = model.load_from_checkpoint("/content/drive/MyDrive/lightning_logs/version_0/checkpoints/epoch=99-step=5000.ckpt")
#torch.save(newmodel.state_dict(), 'dprnn.pth')

# PITLossWrapper works with any loss function.
loss = PITLossWrapper(pairwise_neg_sisdr, pit_from="pw_mtx")

optimizer = optim.Adam(model.parameters(), lr=1e-3)

system = System(model, optimizer, loss, train_loader, val_loader)

  0%|          | 0.00/611M [00:00<?, ?B/s]

Drop 0 utterances from 800 (shorter than 3 seconds)
Drop 0 utterances from 200 (shorter than 3 seconds)


In [ ]:
import torch
# upload the lightning_logs folder about the trained model to colab
# use the best checkpoint to reload our model
# state_dict = torch.load('/content/asteroid/lightning_logs/version0/checkpoints/epoch=99-step=5000.ckpt')
state_dict = torch.load('/content/asteroid/lightning_logs/version1/checkpoints/epoch=99-step=5000.ckpt')
system.load_state_dict(state_dict=state_dict["state_dict"])
system.cpu()

to_save = system.model.serialize()
train_set = LibriMix(
    csv_dir="./MiniLibriMix/metadata/train",
    task= "sep_noisy",
    sample_rate=8000,
    n_src=2,
    segment=4,
)
to_save.update(train_set.get_infos())
# .pth file is used to test the model 
torch.save(to_save, "dprnn_model_22.pth")

#download dprnn_model_22.pth then use the same method as Svoice

Drop 0 utterances from 800 (shorter than 1 seconds)


In [ ]:
!wget https://www.merl.com/demos/deep-clustering/media/male-male-mixture.wav

model.separate("male-male-mixture.wav", force_overwrite=True)

--2022-12-01 16:41:39--  https://www.merl.com/demos/deep-clustering/media/male-male-mixture.wav
Resolving www.merl.com (www.merl.com)... 137.203.190.5
Connecting to www.merl.com (www.merl.com)|137.203.190.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83328 (81K) [audio/x-wav]
Saving to: ‘male-male-mixture.wav’

male-male-mixture.w 100%[===================>]  81.38K  --.-KB/s    in 0.1s    

2022-12-01 16:41:40 (594 KB/s) - ‘male-male-mixture.wav’ saved [83328/83328]



## model separate

In [ ]:
from IPython.display import display, Audio

display(Audio("male-male-mixture_est1.wav"))
display(Audio("male-male-mixture_est2.wav"))

In [ ]:
display(Audio("male-male-mixture.wav"))
